<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AGENTICAI_GEMINI3PRO_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import jax
from jax import numpy as jnp
from warnings import filterwarnings
filterwarnings('ignore')

# --- JAX Hardware Check ---
print("--- Hardware Check (JAX) ---")

# Determine the allocated device
try:
    device = jax.default_backend()
    device_kind = jax.devices()[0].device_kind

    print(f"JAX Default Backend: **{device.upper()}**")
    print(f"JAX Device Kind: **{device_kind}**")

    # Check for specific hardware types
    if 'tpu' in device:
        print("✅ **TPU Runtime Detected and Configured for JAX.**")
    elif 'gpu' in device:
        print("✅ **GPU Runtime Detected and Configured for JAX.**")
    else:
        print("🟡 **CPU Runtime Detected.** (Switch runtime to GPU/TPU for acceleration)")

    # Example JAX Calculation
    key = jax.random.PRNGKey(0)
    x = jax.random.normal(key, (1000, 1000), dtype=jnp.float32)
    y = jnp.dot(x, x)
    print(f"JAX calculation performed on {device_kind}. Result shape: {y.shape}")

except Exception as e:
    print(f"❌ JAX Configuration Error: {e}")

print("----------------------------")

--- Hardware Check (JAX) ---
JAX Default Backend: **TPU**
JAX Device Kind: **TPU v5 lite**
✅ **TPU Runtime Detected and Configured for JAX.**
JAX calculation performed on TPU v5 lite. Result shape: (1000, 1000)
----------------------------


In [3]:
import os
import time
from google.colab import userdata
from google import genai
from google.genai import types
from google.genai.errors import APIError

# --- GEMINI CLIENT SETUP (FIXED by REMOVING client.models.get) ---
try:
    GOOGLE_API_KEY = userdata.get('GEMINI')
    if not GOOGLE_API_KEY:
        raise ValueError("Error: 'GEMINI' API key not found in Colab Secrets.")

    # Initialize the client. This is the only necessary step.
    client = genai.Client(api_key=GOOGLE_API_KEY)
    GEMINI_MODEL_ID = 'gemini-3-pro-preview'

    # **CRITICAL FIX**: Removed the client.models.get() call, which was causing
    # the persistent "Models.get() takes 1 positional argument but 2 were given" error.
    # The client only needs the model ID string for the chat/generate calls.
    print(f"Gemini client successfully configured for model: {GEMINI_MODEL_ID}.")

except Exception as e:
    print(f"Configuration Error: {e}")
    raise SystemExit("Exiting script due to configuration failure.")


# --- 1. A2A PROTOCOL SIMULATION (Delegation Tool) ---
def check_building_permit_status(city: str, project_type: str) -> str:
    """
    Simulates a call to a remote 'Permit Agent' using the A2A protocol.
    Returns the required permit time and complexity for a given project.
    """
    print(f"\n[A2A DELEGATION: Calling Permit Agent for {city} / {project_type}]")
    time.sleep(1)

    if "san francisco" in city.lower():
        if "kitchen" in project_type.lower():
            return "Permit Agent Report: Permit time is 6-8 weeks, high complexity. Requires structural plans."
        else:
            return "Permit Agent Report: Permit time is 2-4 weeks, medium complexity."
    elif "austin" in city.lower():
        return "Permit Agent Report: Permit time is 1-2 weeks, low complexity. Simple online application available."
    else:
        return "Permit Agent Report: City not in database. Assume standard 4-week review."


# --- 2. ADK AGENT DEFINITION (The Architect/Orchestrator) ---
AGENT_INSTRUCTION = """
You are the **Lead Project Architect**, an expert ADK Agent powered by Gemini 3 Pro.
Your task is to create a project execution plan for a home renovation.
Your process must strictly follow these Antigravity-style rules (THINK, DELEGATE, SYNTHESIZE):
1.  **ANALYZE** the user's request and identify the city and project scope.
2.  **PLAN** the execution steps before generating any final output.
3.  **DELEGATE:** You must use the `check_building_permit_status` tool to get critical external information. This tool simulates a call to a separate, specialized A2A agent.
4.  **SYNTHESIZE:** Use the result from the delegated task to write a final, detailed 4-step project plan that includes: [A. Permit Summary, B. Budget Estimate, C. Timeline, D. Next Steps].
"""

# Configure Gemini 3 Pro for Agentic Use (Fixes applied for system_instruction placement)
agent_config = types.GenerateContentConfig(
    temperature=0.2,
    tools=[check_building_permit_status],
    system_instruction=AGENT_INSTRUCTION # Passed here, inside the config object
)


# --- 3. ADK WORKFLOW EXECUTION using ChatSession ---

def run_agentic_workflow(user_prompt: str):
    """
    Uses ChatSession for robust multi-turn function calling, simulating the
    ADK AgentRunner's state management.
    """
    print("="*60)
    print(f"🔥 Starting Agentic Workflow (Gemini 3 Pro + ADK/Antigravity Concepts)")
    print("="*60)
    print(f"👤 User Query:\n{user_prompt}\n")

    # Initialize the Chat Session
    # Fix: The config (which contains the system_instruction) is passed here.
    chat = client.chats.create(
        model=GEMINI_MODEL_ID,
        config=agent_config,
    )

    # --- Turn 1: User sends message, Model generates Function Call (Delegation) ---
    print("🤖 Architect Agent is Planning & Delegating (Deep Think Mode via Instruction)...")
    response_1 = chat.send_message(user_prompt)

    # Check for delegation
    if not response_1.function_calls:
        print("\n[INFO] Model did not generate a function call. Direct Response:")
        print(response_1.text)
        return

    # --- Turn 2: Execute the Delegation (A2A Simulation) ---
    function_call = response_1.function_calls[0]
    function_name = function_call.name
    function_args = dict(function_call.args)

    # Execute the local Python function that simulates the remote A2A Agent
    function_result = check_building_permit_status(**function_args)

    # --- Turn 3: Model receives A2A result and Synthesizes Final Plan ---
    print("\n🤖 Architect Agent receives A2A result and is Synthesizing Final Plan...")

    # Send the function result back to the chat session
    response_2 = chat.send_tool_response(
        function_response=types.FunctionResponse(
            name=function_name,
            response={'report': function_result}
        )
    )

    print("\n" + "="*60)
    print("✨ FINAL PROJECT EXECUTION PLAN (Synthesized by Agent)")
    print("="*60)
    print(response_2.text)


# --- DEMO EXECUTION ---
user_request = "I want a complete project plan to remodel my kitchen in San Francisco. My budget is $75,000."
run_agentic_workflow(user_request)

Gemini client successfully configured for model: gemini-3-pro-preview.
🔥 Starting Agentic Workflow (Gemini 3 Pro + ADK/Antigravity Concepts)
👤 User Query:
I want a complete project plan to remodel my kitchen in San Francisco. My budget is $75,000.

🤖 Architect Agent is Planning & Delegating (Deep Think Mode via Instruction)...

[A2A DELEGATION: Calling Permit Agent for San Francisco / kitchen remodel]

[INFO] Model did not generate a function call. Direct Response:
**Lead Project Architect Execution Plan**

**Project:** Kitchen Remodel
**Location:** San Francisco, CA
**Budget:** $75,000

Based on your request and the data retrieved from our Permit Agent, I have developed the following execution strategy. San Francisco is a high-complexity jurisdiction; strict adherence to this plan is required to stay on budget.

### A. Permit Summary (Critical Path)
*   **Status:** High Complexity.
*   **Estimated Wait Time:** 6-8 weeks.
*   **Key Requirement:** The Permit Agent indicates that **struc